In [1]:
import numpy as np
import cv2 as cv
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
train = 'Data/Train'
test = 'Data/Test'
validation = 'Data/Validation'

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255, zoom_range=0.1, shear_range=0.2)
train_datagen = datagen.flow_from_directory(directory=train, target_size=(128,128), batch_size=32, class_mode='binary')
test_datagen = datagen.flow_from_directory(directory=test, target_size=(128,128), batch_size=32, class_mode='binary')
validation_datagen = datagen.flow_from_directory(directory=validation, target_size=(128,128), batch_size=32, class_mode='binary')

Found 10000 images belonging to 2 classes.
Found 988 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


Rescale:
To normalize the pixel range, which aids in neural network training

Data Augmentation:
zoom_range:
how zoomed the image is

shear_range:
angle augmentation

Batch size vs Epochs:
The batch size is a hyperparameter of gradient descent that controls the number of training samples to work through before the model’s internal parameters are updated.
The number of epochs is a hyperparameter of gradient descent that controls the number of complete passes through the training dataset.


In [4]:
base_model = tf.keras.applications.densenet.DenseNet201(weights = 'imagenet', input_shape = (128, 128, 3), include_top = False)
for layer in base_model.layers:
    layer.trainable = False

In [5]:
model = tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation = 'relu'))
# Dropout can be added if necessary to avoid overfitting
# Dropout is a technique where randomly selected neurons are ignored during training.
# model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(100, activation = 'relu'))
# model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 4, 4, 1920)        18321984  
_________________________________________________________________
flatten (Flatten)            (None, 30720)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               3072100   
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 21,404,285
Trainable params: 3,082,301
Non-trainable params: 18,321,984
_________________________________________________________________


In [6]:
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(0.001, 10000, 0.00001,power=0.4)
model.compile(  loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                metrics = ['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                min_delta = 0.005,
                                patience = 5,
                                restore_best_weights = True)

In [8]:
history = model.fit(train_datagen,
                    validation_data = validation_datagen,
                    epochs = 40,
                    callbacks = [callback],
                    verbose = 1)

Epoch 1/40
313/313 [==============================] - 562s 2s/step - loss: 0.0651 - accuracy: 0.9898 - val_loss: 0.0072 - val_accuracy: 0.9975
Epoch 2/40
313/313 [==============================] - 588s 2s/step - loss: 0.0142 - accuracy: 0.9968 - val_loss: 0.0446 - val_accuracy: 0.9912
Epoch 3/40
313/313 [==============================] - 560s 2s/step - loss: 0.0075 - accuracy: 0.9976 - val_loss: 0.0362 - val_accuracy: 0.9962
Epoch 4/40
313/313 [==============================] - 560s 2s/step - loss: 0.0141 - accuracy: 0.9970 - val_loss: 2.0016e-05 - val_accuracy: 1.0000
Epoch 5/40
313/313 [==============================] - 545s 2s/step - loss: 0.0102 - accuracy: 0.9979 - val_loss: 5.8576e-04 - val_accuracy: 1.0000
Epoch 6/40
313/313 [==============================] - 530s 2s/step - loss: 2.3646e-04 - accuracy: 0.9999 - val_loss: 0.0110 - val_accuracy: 0.9975


In [9]:
model.evaluate(test_datagen)

31/31 [==============================] - 54s 2s/step - loss: 0.0393 - accuracy: 0.9949


[0.0392596572637558, 0.9949392676353455]

In [13]:
tzuyu = plt.imread('tzuyu.jpg')
tzuyu_mask = plt.imread('tzuyu_mask.jpg')
# imagedatagenerator/plt.imshow uses RGB
# OpenCV uses BGR
tzuyu = cv.resize(tzuyu, (128,128), interpolation = cv.INTER_CUBIC)
tzuyu_mask = cv.resize(tzuyu_mask, (128,128), interpolation = cv.INTER_CUBIC)
ty_predict = np.array(tzuyu / 255)
ty_mask_predict = np.array(tzuyu_mask / 255)

In [14]:
model.predict(np.array([ty_predict]))

array([[1.]], dtype=float32)

In [16]:
model.predict(np.array([ty_mask_predict]))

array([[9.129589e-16]], dtype=float32)

In [17]:
model.save('mask_detection_128x128.h5')